In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pyathena import connect
import pandas as pd
import json
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
##pd.set_option('display.max_colwidth', None)
pd.options.display.max_colwidth = 200
pd.options.display.float_format = '{:.2f}'.format

conn = connect(s3_staging_dir='s3://jm-dsprod-athena/data-scientist-workgroup', region_name='ap-south-1',)

%load_ext athena

The athena extension is already loaded. To reload it, use:
  %reload_ext athena


__Final data set for analysis__<br>
ltv_final_<br>
after date filter: ltv_final1

In [17]:
%%athena
select * from ltv_final1 limit 3

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,17b44e6b-24f4-463b-a4ba-a80d4bf8060b,2021-12-01,2010-01-01,0,googleadwords_int,2021-12-01,0,0.00,500.00,500.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
1,b97c2766-6df1-4e15-aca4-bc9091294e47,2022-03-01,2022-03-01,1,creditkaro_int,2022-03-01,0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
2,37e779bd-040e-495d-8ea3-114b51a20688,2022-05-01,2010-01-01,0,phonepe_int,2022-05-01,0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00


In [4]:
%%athena
select rewards_expense/monthly_expense as rewards_expense_percent
, domestic_non_atm_expense
select ltv_month
, sum(domestic_interchange_revenue) as domestic_interchange_revenue
, sum(intl_interchange_revenue) as intl_interchange_revenue
, sum(monthly_sum_daily_pf_revenue) as account_balance_revenue
, sum(total_rewards_cost) as rewards_expense
, sum(domestic_atm_cost) as domestic_atm_expense
, sum(domestic_non_atm_cost) as doemstic_non_atm_expense
, sum(intl_atm_cost) as intl_atm_expense
, sum(intl_non_atm_cost) as intl_non_atm_expense
, sum(month_net_expense) as monthly_expense
, sum(month_net_revenue) as monthly_revenue
, sum(month_net_clv) as monthly_clv
from ltv_final1
group by 1
order by 1

,ltv_month,domestic_interchange_revenue,intl_interchange_revenue,account_balance_revenue,rewards_expense,domestic_atm_expense,doemstic_non_atm_expense,intl_atm_expense,intl_non_atm_expense,monthly_expense,monthly_revenue,monthly_clv
0,2021-08-01,478824.01,17758.92,2011.85,3343852.00,5402.16,135750.44,0.00,46221.86,3531226.46,498594.78,-3032631.68
1,2021-09-01,458057.52,25788.61,7689.40,3680682.08,29690.28,99891.14,717.26,65241.32,3876222.08,491535.52,-3384686.57
2,2021-10-01,1194050.53,100045.67,19456.27,6267371.00,83032.20,253504.19,5031.70,247206.31,6856145.40,1313552.47,-5542592.94
3,2021-11-01,1813381.26,182917.20,39453.69,8948963.00,143918.10,363213.45,2803.12,468739.77,9927637.44,2035752.15,-7891885.29
4,2021-12-01,2899770.51,311476.51,71943.37,12702919.00,296486.46,499721.59,8975.65,787169.89,14295272.59,3283190.38,-11012082.21
5,2022-01-01,3279241.73,570124.61,118862.15,8900267.00,388978.47,499491.57,12405.74,1451374.37,11252517.15,3968228.48,-7284288.67
6,2022-02-01,4313497.95,670554.44,169922.58,8494882.00,533252.39,630638.37,23713.10,1683134.00,11365619.86,5153974.97,-6211644.89
7,2022-03-01,5649713.86,767893.72,270847.43,10491646.00,790770.78,713147.04,196550.19,1483530.45,13675644.46,6688455.02,-6987189.44
8,2022-04-01,6157989.87,1207201.05,350609.69,10366100.00,915295.37,712059.41,600827.97,1567446.49,14161729.23,7715800.60,-6445928.63
9,2022-05-01,6935070.72,2124022.61,446366.83,14316823.00,955496.61,893948.59,1261810.53,2221464.23,19649542.96,9505460.16,-10144082.79


In [7]:
%%athena
select count(distinct user_id) as uc
from ltv_final1

,uc
0,1090438


In [5]:
%%athena
with txn_latest_update as (
    select rwd_txn_id, max(updated_ts) as max_update_ts
    from transformations_db.jupiter_rewards_transaction_master
    where is_deleted = False
    group by 1
)

, txn_base as (
    select * from transformations_db.jupiter_rewards_transaction_master
)

, latest_base as (
    select a.*
    from txn_base a 
    join txn_latest_update b
    on a.rwd_txn_id = b.rwd_txn_id
    and a.updated_ts = b.max_update_ts
    where txn_status = 'completed'
)

, rwd_base as (
    select user_id
    , date(txn_ts) as txn_date
    , txn_id
    , jupiter_instrument
    , action
    , txn_type
    , txn_amount
    from latest_base 
    where date(txn_ts) between date('2022-03-01') and date('2022-03-31')
    group by 1,2,3,4,5,6,7
)

select sum(txn_amount) as ta
from rwd_base
where action <> 'convert-to-cash'
and txn_type = 'credit'

,ta
0,14701920.00


In [9]:
%%athena
select sum(dc_txn_amount) as total_amt
, sum(dc_txn_count) as total_txn_cnt
, count(distinct user_id) as user_count
, sum(total_rewards_cost) as rwd_cost
, sum(month_avg_balance) as total_avg_balance
from ltv_final1

,total_amt,total_txn_cnt,user_count,rwd_cost,total_avg_balance
0,9379429897.39,4596731,1090438,87513505.08,3277253431.49


__average numbers for revenue and expenses__

In [10]:
%%athena
select * from user_rwd_agg limit 3

,user_id,update_date,total_rewards_cost
0,c9e938b8-0998-4b95-b25a-66f87166021c,2022-05-30,250.00
1,82575f06-fe19-475d-8177-f5fb0d84626d,2022-04-02,5.00
2,9pX-cjm-kzxf-Wjg,2022-02-04,44.00


In [11]:
%%athena
select * from dc_base limit 3

,user_id,txn_id,txn_date,txn_type,dc_txn_type,dc_txn_scope,txn_amount
0,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,63515932-45a9-4a29-8491-f19117784ef2,2022-01-01,WITHDRAWAL,Non_ATM,DOMESTIC,180.00
1,6lC-xKq-C0OR-sM1,65acd487-9dc8-48a4-b3bd-ca4f515e2089,2022-01-01,WITHDRAWAL,ATM,DOMESTIC,400.00
2,d6bf6ccc-1026-4461-a5bc-80b2c6b233e2,1ae53981-0277-40a5-abdf-8460c2c8d480,2022-01-01,WITHDRAWAL,Non_ATM,DOMESTIC,86.00


In [10]:
%%athena
with dc_base as (
    select * from dc_base
    where date(txn_date) between date('2021-08-01') and date('2022-05-31')
)

, base as (
    select dc_txn_scope, dc_txn_type, count(distinct txn_id) as tc, sum(txn_amount) as ta
    from dc_base
    group by 1,2
)


select * from base order by 1,2

,dc_txn_scope,dc_txn_type,tc,ta
0,DOMESTIC,ATM,1195056,4602585600.00
1,DOMESTIC,Non_ATM,3299263,4364882499.82
2,INTERNATIONAL,ATM,4187,145712776.16
3,INTERNATIONAL,Non_ATM,96936,263724439.07
4,None,ATM,130,514000.00
5,None,Non_ATM,1182,2025576.06


In [15]:
df_ = pd.read_sql('''
with dc_base as (
    select * from dc_base
    where date(txn_date) between date('2021-08-01') and date('2022-05-31')
)

, base as (
    select user_id, count(distinct txn_id) as tc, sum(txn_amount) as ta
    from dc_base
    where dc_txn_scope = 'DOMESTIC'
    group by 1
)


select user_id, sum(ta)/(sum(tc)*1.0) as txn_tkt
from base
group by 1''', conn)
df_.describe()

,txn_tkt
count,398747.00
mean,1808.02
std,3318.35
min,0.43
25%,352.68
50%,697.50
75%,2000.00
max,297477.70


In [16]:
df_1 = pd.read_sql('''
with dc_base as (
    select * from dc_base
    where date(txn_date) between date('2021-08-01') and date('2022-05-31')
)

, base as (
    select user_id, count(distinct txn_id) as tc, sum(txn_amount) as ta
    from dc_base
    where dc_txn_scope = 'INTERNATIONAL'
    group by 1
)


select user_id, sum(ta)/(sum(tc)*1.0) as txn_tkt
from base
group by 1''', conn)
df_1.describe()

,txn_tkt
count,19940.00
mean,2361.32
std,8780.12
min,0.62
25%,28.00
50%,196.67
75%,964.57
max,270144.64


In [26]:
%%athena
select count(distinct user_id) as uc
, count(distinct txn_id) as tc
, sum(txn_amount) as ta
from dc_base
where dc_txn_scope = 'DOMESTIC'
and dc_txn_type = 'ATM'

,uc,tc,ta
0,186222,1271668,4899176600.00


In [14]:
%%athena
select * from jupiter_eod_balance 
where user_id = 'cf7f3d20-1bef-4f9f-999a-06c16eb3e32e'
order by update_date

,user_id,update_date,jupiter_balance,n_days,n_days_balance,daily_avg_balance,ob_date,fkyc_date
0,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-03,500.00,1,500.00,500.00,2021-11-02,2021-11-15
1,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-04,0.00,2,500.00,250.00,2021-11-02,2021-11-15
2,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-05,0.00,3,500.00,166.67,2021-11-02,2021-11-15
3,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-06,0.00,4,500.00,125.00,2021-11-02,2021-11-15
4,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-07,0.00,5,500.00,100.00,2021-11-02,2021-11-15
5,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-08,0.00,6,500.00,83.33,2021-11-02,2021-11-15
6,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-09,0.00,7,500.00,71.43,2021-11-02,2021-11-15
7,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-10,2000.00,8,2500.00,312.50,2021-11-02,2021-11-15
8,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-11,2000.00,9,4500.00,500.00,2021-11-02,2021-11-15
9,cf7f3d20-1bef-4f9f-999a-06c16eb3e32e,2021-11-12,2000.00,10,6500.00,650.00,2021-11-02,2021-11-15


In [18]:
%%athena
select approx_percentile(max_n_days, 0.5) as median_days
from(
select user_id, max(n_days) as max_n_days, sum(jupiter_balance) as sum_jupiter_bal
from jupiter_eod_balance
--where user_id = 'cf7f3d20-1bef-4f9f-999a-06c16eb3e32e'
group by 1 
)

,median_days
0,124


In [21]:
%%athena
select sum(sum_daily_avg_balance) as sum_daily_avg_balance, sum(max_n_days) as sum_days, count(distinct user_id) as uc
from(
select user_id, max(n_days) as max_n_days, sum(daily_avg_balance) as sum_daily_avg_balance
from jupiter_eod_balance
--where user_id = 'cf7f3d20-1bef-4f9f-999a-06c16eb3e32e'
group by 1 
)

,sum_daily_avg_balance,sum_days,uc
0,98084548225.28,108083111,867463


In [22]:
%%athena
select approx_percentile(sum_daily_avg_balance, 0.5) as median_sum_daily_avg_balance, sum(max_n_days) as sum_days, count(distinct user_id) as uc
from(
select user_id, max(n_days) as max_n_days, sum(daily_avg_balance) as sum_daily_avg_balance
from jupiter_eod_balance
--where user_id = 'cf7f3d20-1bef-4f9f-999a-06c16eb3e32e'
group by 1 
)

,median_sum_daily_avg_balance,sum_days,uc
0,11135.999,108083111,867463


In [25]:
%%athena
select count(distinct user_id) as uc
from(
    select user_id, sum(daily_avg_balance) as sum_daily_avg_balance
    from jupiter_eod_balance
    group by 1
)
where sum_daily_avg_balance > 0

,uc
0,785516


In [23]:
%%athena
select user_id, sum(daily_avg_balance) as sum_daily_avg_balance
from jupiter_eod_balance
group by 1
order by 2 desc 
limit 50

,user_id,sum_daily_avg_balance
0,f365e38a-d62f-4e9f-b985-bbbcae899fd5,290350089.56
1,9bcbede3-50b1-472c-b787-3a04a2e6089e,189266186.90
2,5f815a94-2ab9-4b13-95a5-1feb4bcba27f,98199689.35
3,cb21c583-7ace-4913-9a34-aa6eadfe4e5d,88978882.50
4,2792b617-9932-4f07-af55-299a0296111c,88395480.95
5,9256a058-8e30-4c67-9091-0871ce5848d7,81783699.72
6,b97c6365-1143-4245-9cc9-2ac2adb125a7,75152030.22
7,zPK-18D-PJVm-9yA,66951634.40
8,1641bf32-0a59-44d3-bca7-1f505b4bd1ee,64111127.16
9,f1826e0d-438b-4b4e-acc3-6383296779d2,58954050.62


In [24]:
%%athena
select user_id, sum(daily_avg_balance) as sum_daily_avg_balance
from jupiter_eod_balance
group by 1
order by 2
limit 10

,user_id,sum_daily_avg_balance
0,5bc71888-fdc0-487a-ab5a-532879cd2359,0.00
1,198f8ef7-cb82-4682-9264-8d661d8d122f,0.00
2,1FL-Xlr-Yie9-u12,0.00
3,19250427-880c-4471-92a3-f526b24c0806,0.00
4,19d4fcf1-83a7-4f3d-afd7-b202f8c0dd1e,0.00
5,0f2804c2-76d0-494d-a15d-28acc6ce205e,0.00
6,18a41ac9-d01f-43d7-9d78-c6de8ce4b0a4,0.00
7,5c4cee98-d6c9-4f69-a7dd-4492ebcef64e,0.00
8,5c11a75d-acff-4118-adea-151c8b3ebe7b,0.00
9,19c16d8e-e2a5-4f6c-8113-7eb4915559e1,0.00


In [4]:
%%athena
select * from ltv_final_
where user_id = 'e12aff58-cbf9-477e-8f75-d763367305b6'
order by ltv_month

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,e12aff58-cbf9-477e-8f75-d763367305b6,2022-01-01,2022-01-01,1,organic,2022-01-01,0,0.00,201.00,250.83,0.13,0.01,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.01,-9.99
1,e12aff58-cbf9-477e-8f75-d763367305b6,2022-01-01,2022-01-01,0,organic,2022-02-01,2,600.00,2416.00,5257.07,1.51,3.02,7.00,0.00,0.66,0.00,0.00,2.22,0.00,1.56,7.66,5.24,-2.42
2,e12aff58-cbf9-477e-8f75-d763367305b6,2022-01-01,2022-01-01,0,organic,2022-03-01,0,0.00,65.00,2640.39,0.04,1.68,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.68,1.68
3,e12aff58-cbf9-477e-8f75-d763367305b6,2022-01-01,2022-01-01,0,organic,2022-04-01,0,0.00,229.75,154.82,0.14,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.10
4,e12aff58-cbf9-477e-8f75-d763367305b6,2022-01-01,2022-01-01,0,organic,2022-05-01,0,0.00,2669.75,822.65,1.67,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.52,0.52


In [5]:
%%athena
select ltv_month, count(distinct user_id) as user_count
, sum(eom_balance) as eom_balance
, sum(month_avg_balance) as month_avg_balance
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_net_expense) as month_net_expense
, sum(month_net_revenue) as month_net_revenue
, sum(month_net_clv) as month_net_clv
from ltv_final_
group by 1
order by 1

,ltv_month,user_count,eom_balance,month_avg_balance,dc_txn_count,dc_txn_amount,month_net_expense,month_net_revenue,month_net_clv
0,2020-09-01,1,0.00,0.00,0,0.00,2.08,0.00,-2.08
1,2021-05-01,17,0.00,0.00,55,5942.75,61.97,3.73,-58.24
2,2021-06-01,332,0.00,0.00,332,451534.98,18792.49,1654.33,-17138.16
3,2021-07-01,6128,0.00,0.00,8767,18191147.04,570167.27,66950.80,-503216.46
4,2021-08-01,18529,15372369.19,14022679.39,40866,130724774.07,3531226.46,498594.78,-3032631.68
5,2021-09-01,53201,28433994.09,25342487.17,54868,127799864.78,3876222.08,491535.52,-3384686.57
6,2021-10-01,113528,69066367.63,60697932.72,136216,329771870.93,6856145.40,1313552.47,-5542592.94
7,2021-11-01,180203,144214688.47,122540891.38,267375,502631617.81,9927637.44,2035752.15,-7891885.29
8,2021-12-01,222422,248921735.51,197825591.45,415912,805055766.51,14295272.59,3283190.38,-11012082.21
9,2022-01-01,274484,304721216.74,287291625.23,476943,925331179.67,11252517.15,3968228.48,-7284288.67


In [8]:
%%athena
select count(distinct user_id) 
from(
    select user_id, sum(month_net_clv) as clv
    from ltv_final_ 
    group by 1
)
where clv > 0 

,_col0
0,96379


In [5]:
%%athena
select min(clv) as min_c, max(clv) as max_c
from(
    select user_id, sum(month_net_clv)  as clv
    from ltv_final_
    group by 1
)

,min_c,max_c
0,-253600.38,40943.93


In [9]:
%%athena
drop table if exists ltv_final1

In [10]:
%%athena
create table ltv_final1 as 
select * from ltv_final_
where ltv_month >= date('2021-08-01') and ltv_month <= date('2022-05-01')

In [11]:
%%athena
select * from ltv_final1 limit 3

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,17b44e6b-24f4-463b-a4ba-a80d4bf8060b,2021-12-01,2010-01-01,0,googleadwords_int,2021-12-01,0,0.00,500.00,500.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
1,b97c2766-6df1-4e15-aca4-bc9091294e47,2022-03-01,2022-03-01,1,creditkaro_int,2022-03-01,0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
2,37e779bd-040e-495d-8ea3-114b51a20688,2022-05-01,2010-01-01,0,phonepe_int,2022-05-01,0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00


In [13]:
df_ltv_mo = pd.read_sql('''select ltv_month, sum(month_net_clv) as clv
from ltv_final1
group by 1''', conn)

In [14]:
df_ltv_mo.head()

,ltv_month,clv
0,2021-11-01,-7891885.29
1,2021-12-01,-11012082.21
2,2021-09-01,-3384686.57
3,2022-02-01,-6211644.89
4,2021-08-01,-3032631.68


In [15]:
df_ltv_mo.describe()

,clv
count,10.00
mean,-6793701.31
std,2545059.47
min,-11012082.21
25%,-7739986.13
50%,-6716559.03
75%,-5709855.92
max,-3032631.68


In [16]:
df_ltv_user = pd.read_sql('''
select user_id, sum(month_net_clv) as net_clv
from ltv_final1
group by 1''', conn)

In [17]:
df_ltv_user.head()

,user_id,net_clv
0,346bf49c-edba-47af-873f-c5259ba50e3c,-447.19
1,1e78b9d1-ea31-4989-8d29-af56bcae48ab,-10.00
2,973e20d8-100b-4376-8458-03efc2c0d0ad,-75.38
3,bd3ac610-750f-43ec-bb68-46c3484bc57f,-9.24
4,2ab99a2d-d636-4fd3-aa7d-a899539cedcb,2.35


In [18]:
df_ltv_user.describe()

,net_clv
count,1090438.00
mean,-62.30
std,597.18
min,-253598.38
25%,-15.00
50%,-10.00
75%,-10.00
max,39303.32


In [21]:
%%athena
select ltv_bin, count(distinct user_id) as user_count
from(
    select user_id
    , case when ltv < -50000 then '-50K+'
    when ltv >= -50000 and ltv < -10000 then '[-50K,-10K)'
    when ltv >= -10000 and ltv < -1000 then '[-10K,-1K)'
    when ltv >= -1000 and ltv < -100 then '[-1K,-100)'
    when ltv >= -100 and ltv < -10 then '[-100, -10)'
    when ltv >= -10 and ltv < 0 then '[-10,0)'
    when ltv >= 0 and ltv < 10 then '[0,10)'
    when ltv >= 10 and ltv < 100 then '[10,100)'
    when ltv >= 100 and ltv < 1000 then '[100, 1K)'
    when ltv >= 1000 and ltv < 10000 then '[1K, 10K)'
    when ltv >= 10000 and ltv < 50000 then '[10K, 50K)'
    when ltv >= 50000 then '50K+'
    end as ltv_bin
    from(
        select user_id, sum(month_net_clv) as ltv
        from ltv_final1
        group by 1
    )
    group by 1,2
)
group by 1

,ltv_bin,user_count
0,"[-100, -10)",215005
1,"[-50K,-10K)",103
2,-50K+,10
3,"[10K, 50K)",4
4,"[-10,0)",679174
5,"[-1K,-100)",87732
6,"[0,10)",23508
7,"[1K, 10K)",889
8,"[10,100)",46523
9,"[100, 1K)",22898


In [22]:
%%athena
select ltv_month, ltv_bin, count(distinct user_id) as user_count
from(
    select ltv_month
    ,user_id
    , case when ltv < -50000 then '-50K+'
    when ltv >= -50000 and ltv < -10000 then '[-50K,-10K)'
    when ltv >= -10000 and ltv < -1000 then '[-10K,-1K)'
    when ltv >= -1000 and ltv < -100 then '[-1K,-100)'
    when ltv >= -100 and ltv < -10 then '[-100, -10)'
    when ltv >= -10 and ltv < 0 then '[-10,0)'
    when ltv >= 0 and ltv < 10 then '[0,10)'
    when ltv >= 10 and ltv < 100 then '[10,100)'
    when ltv >= 100 and ltv < 1000 then '[100, 1K)'
    when ltv >= 1000 and ltv < 10000 then '[1K, 10K)'
    when ltv >= 10000 and ltv < 50000 then '[10K, 50K)'
    when ltv >= 50000 then '50K+'
    end as ltv_bin
    from(
        select ltv_month, user_id, sum(month_net_clv) as ltv
        from ltv_final1
        group by 1,2
    )
    group by 1,2,3
)
group by 1,2

,ltv_month,ltv_bin,user_count
0,2021-11-01,"[-1K,-100)",12445
1,2022-04-01,"[10,100)",32311
2,2022-02-01,"[0,10)",95726
3,2021-11-01,"[0,10)",19456
4,2021-12-01,"[-1K,-100)",18241
5,2021-08-01,"[-100, -10)",13325
6,2021-10-01,"[-10K,-1K)",160
7,2021-08-01,"[10,100)",338
8,2021-11-01,"[1K, 10K)",10
9,2022-03-01,"[0,10)",120075


In [23]:
df_ = pd.read_sql('''
select ltv_month, ltv_bin, count(distinct user_id) as user_count
from(
    select ltv_month
    ,user_id
    , case 
    when ltv <= -1000 then '<= -1K'
    when ltv >= -1000 and ltv < -100 then '[-1K,-100)'
    when ltv >= -100 and ltv < -10 then '[-100, -10)'
    when ltv >= -10 and ltv < 0 then '[-10,0)'
    when ltv >= 0 and ltv < 10 then '[0,10)'
    when ltv >= 10 and ltv < 100 then '[10,100)'
    when ltv >= 100 and ltv < 1000 then '[100, 1K)'
    when ltv >= 1000 then '>= 1K'
    end as ltv_bin
    from(
        select ltv_month, user_id, sum(month_net_clv) as ltv
        from ltv_final1
        group by 1,2
    )
    group by 1,2,3
)
group by 1,2
''', conn)
df_.to_csv('ltv_month_bin.csv', index=False)

In [24]:
%%athena
create table user_clv_lt100 as 
select user_id, clv from(
    select user_id, sum(month_net_clv) as clv
    from ltv_final1
    group by 1
)
where clv <= 100

In [25]:
%%athena
create table user_clv_gt100 as 
select user_id, clv from(
    select user_id, sum(month_net_clv) as clv
    from ltv_final1
    group by 1
)
where clv >= 100

In [26]:
%%athena
select * from ltv_final1 limit 10

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,7dbd9d1a-9265-42f6-9b62-be13ff9718eb,2021-12-01,2010-01-01,0,None,2022-02-01,1,526.00,1975.00,1448.57,0.00,0.00,6.00,0.00,0.58,0.00,0.00,1.95,0.00,1.37,6.58,1.95,-4.63
1,1ce687c5-c18d-4684-8f96-c1f2370a4e2b,2021-10-01,2021-10-01,0,Facebook Ads,2022-01-01,3,4500.00,210.72,535.60,0.13,0.34,0.00,4.05,0.00,0.00,0.00,16.65,0.00,12.60,4.05,16.99,12.94
2,aca7724e-95d6-4cdd-8f5c-6a7189fc1e24,2022-04-01,2010-01-01,0,onecode_int,2022-04-01,0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
3,e318b4b6-7b91-456f-b78e-1d7135d95181,2021-08-01,2021-08-01,0,None,2022-01-01,0,0.00,354.95,354.95,0.22,0.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.23,0.23
4,76a4d523-ce1b-476c-9219-eb06e613f915,2021-08-01,2021-09-01,0,None,2022-03-01,0,0.00,51.00,11.48,0.03,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01
5,hsn-lxL-xwxn-Y7s,2021-09-01,2021-09-01,0,googleadwords_int,2022-02-01,0,0.00,100.00,100.00,0.06,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.06
6,d8d945e3-d97a-4419-830e-1ce3c5737a3c,2021-09-01,2021-09-01,0,googleadwords_int,2022-03-01,1,154.50,381.34,100.39,0.24,0.06,2.00,0.00,0.17,0.00,0.00,0.57,0.00,0.40,2.17,0.64,-1.53
7,387c0507-b6a9-4f19-a064-c428896e7aec,2022-05-01,2022-05-01,1,Facebook Ads,2022-05-01,0,0.00,100.00,100.00,0.06,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
8,96f21a10-4d25-4080-9a19-0b879c0b2a4c,2021-12-01,2022-01-01,0,restricted,2022-02-01,1,128.00,15422.20,4471.83,9.64,2.57,80.00,0.00,0.14,0.00,0.00,0.47,0.00,0.33,80.14,3.05,-77.09
9,82d6b22c-3574-4b1e-aaa3-b3c019db867b,2022-04-01,2010-01-01,0,Facebook Ads,2022-04-01,0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00


In [11]:
%%athena
select ob_month
, count(distinct user_id) as user_count
, sum(ltv_month_count) as ltv_month_count
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_avg_balance) as month_avg_balance
from(
    select a.user_id, ob_month, media_source
    , count(distinct ltv_month) as ltv_month_count
    , sum(dc_txn_count) as dc_txn_count
    , sum(dc_txn_amount) as dc_txn_amount
    , sum(month_avg_balance) as month_avg_balance
    from ltv_final1 a 
    join user_clv_gt100 b
    on a.user_id = b.user_id
    group by 1,2,3
)
group by 1
order by 1

,ob_month,user_count,ltv_month_count,dc_txn_count,dc_txn_amount,month_avg_balance
0,2021-05-01,2,20,238,1973695.77,13653030.63
1,2021-06-01,15,113,353,997676.47,12840117.96
2,2021-07-01,115,852,4073,19477573.37,10145895.49
3,2021-08-01,255,1759,9300,47245275.56,21635148.58
4,2021-09-01,1544,9912,54110,257494782.22,51573855.92
5,2021-10-01,2988,19351,96588,408883983.37,83981902.93
6,2021-11-01,4427,25886,133766,591162168.72,148043348.07
7,2021-12-01,3938,20658,107938,486890278.11,119408761.59
8,2022-01-01,3900,17358,103208,502389418.88,122059456.54
9,2022-02-01,3583,12319,100356,484784446.21,80204047.62


In [6]:
%%athena
select media_source
, count(distinct user_id) as user_count
, sum(ltv_month_count) as ltv_month_count
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_avg_balance) as month_avg_balance
from(
    select a.user_id, ob_month, media_source
    , count(distinct ltv_month) as ltv_month_count
    , sum(dc_txn_count) as dc_txn_count
    , sum(dc_txn_amount) as dc_txn_amount
    , sum(month_avg_balance) as month_avg_balance
    from ltv_final1 a 
    join user_clv_gt100 b
    on a.user_id = b.user_id
    group by 1,2,3
)
where media_source in ('organic', 'googleadwords_int', 'restricted', 'onecode_int', 'Facebook Ads'
                      , 'None', 'Partnerships', 'Virality', 'phonepe_int', 'creditkaro_int', 'gpay_code'
                      , 'Jupiter_Referral', 'website', 'appnext_int', 'Twitter', 'Amazon', 'Influencers')
group by 1
order by 1

,media_source,user_count,ltv_month_count,dc_txn_count,dc_txn_amount,month_avg_balance
0,Amazon,200,826,4159,22816507.12,2083886.43
1,Facebook Ads,1732,10622,58954,229975568.59,30476147.07
2,Influencers,104,565,1866,9278174.48,7989754.20
3,Jupiter_Referral,7,7,105,940400.00,14872.17
4,None,4,21,81,328520.73,38232.36
5,Partnerships,470,1998,14136,57659024.71,4561354.30
6,Twitter,254,1195,7097,29542431.98,8846538.63
7,Virality,956,4560,24753,112726688.08,30482058.94
8,appnext_int,117,313,3379,13444936.42,2189360.79
9,creditkaro_int,411,2110,12045,60273159.99,5813509.26


In [12]:
%%athena
select ob_month
, count(distinct user_id) as user_count
, sum(ltv_month_count) as ltv_month_count
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_avg_balance) as month_avg_balance
from(
    select a.user_id, ob_month, media_source
    , count(distinct ltv_month) as ltv_month_count
    , sum(dc_txn_count) as dc_txn_count
    , sum(dc_txn_amount) as dc_txn_amount
    , sum(month_avg_balance) as month_avg_balance
    from ltv_final1 a 
    join user_clv_lt100 b
    on a.user_id = b.user_id
    group by 1,2,3
)
group by 1
order by 1

,ob_month,user_count,ltv_month_count,dc_txn_count,dc_txn_amount,month_avg_balance
0,2021-05-01,25,225,1984,3017258.35,6761884.90
1,2021-06-01,231,1808,8519,12309038.42,21970313.97
2,2021-07-01,4050,29961,124623,256423714.96,90933002.19
3,2021-08-01,15502,58488,172926,483373103.88,82862224.23
4,2021-09-01,55003,153263,306007,570209859.78,155047303.74
5,2021-10-01,80928,244855,503396,907415869.04,263713347.25
6,2021-11-01,130607,362724,711855,1009784835.21,419966506.77
7,2021-12-01,129066,337741,563214,878274227.62,370421340.48
8,2022-01-01,139839,320923,496382,703049839.46,378962551.06
9,2022-02-01,117130,261600,486016,629437870.17,330149341.02


In [13]:
%%athena
select media_source
, count(distinct user_id) as user_count
, sum(ltv_month_count) as ltv_month_count
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_avg_balance) as month_avg_balance
from(
    select a.user_id, ob_month, media_source
    , count(distinct ltv_month) as ltv_month_count
    , sum(dc_txn_count) as dc_txn_count
    , sum(dc_txn_amount) as dc_txn_amount
    , sum(month_avg_balance) as month_avg_balance
    from ltv_final1 a 
    join user_clv_lt100 b
    on a.user_id = b.user_id
    group by 1,2,3
)
where media_source in ('organic', 'googleadwords_int', 'restricted', 'onecode_int', 'Facebook Ads'
                      , 'None', 'Partnerships', 'Virality', 'phonepe_int', 'creditkaro_int', 'gpay_code'
                      , 'Jupiter_Referral', 'website', 'appnext_int', 'Twitter', 'Amazon', 'Influencers')
group by 1
order by 1

,media_source,user_count,ltv_month_count,dc_txn_count,dc_txn_amount,month_avg_balance
0,Amazon,11532,23803,35336,47408177.49,24512545.50
1,Facebook Ads,71504,154980,250222,345837121.30,124882501.25
2,Influencers,7767,22179,36172,44542706.71,47860079.58
3,Jupiter_Referral,13747,14087,11011,18009229.34,8098354.42
4,None,326,1128,2909,5486278.58,3112979.76
5,Partnerships,48581,73350,60087,103753471.56,30846307.48
6,Twitter,11795,28906,54568,66818366.31,40978589.64
7,Virality,36315,103614,215118,500476361.01,129439293.63
8,appnext_int,12064,16738,18806,22438674.41,13064963.46
9,creditkaro_int,29184,56794,68183,112343560.52,27265396.51


In [5]:
%%athena
select count(distinct user_id) as user_count
, sum(ltv_month_count) as ltv_month_count
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_avg_balance) as month_avg_balance
from(
    select a.user_id
    , count(distinct ltv_month) as ltv_month_count
    , sum(dc_txn_count) as dc_txn_count
    , sum(dc_txn_amount) as dc_txn_amount
    , sum(month_avg_balance) as month_avg_balance
    from ltv_final1 a 
    join user_clv_lt100 b
    on a.user_id = b.user_id
    group by 1
)

,user_count,ltv_month_count,dc_txn_count,dc_txn_amount,month_avg_balance
0,1066647,2295560,3930358,6187466208.80,2560774550.39


In [6]:
%%athena
select count(distinct user_id) as user_count
, sum(ltv_month_count) as ltv_month_count
, sum(dc_txn_count) as dc_txn_count
, sum(dc_txn_amount) as dc_txn_amount
, sum(month_avg_balance) as month_avg_balance
from(
    select a.user_id
    , count(distinct ltv_month) as ltv_month_count
    , sum(dc_txn_count) as dc_txn_count
    , sum(dc_txn_amount) as dc_txn_amount
    , sum(month_avg_balance) as month_avg_balance
    from ltv_final1 a 
    join user_clv_gt100 b
    on a.user_id = b.user_id
    group by 1
)

,user_count,ltv_month_count,dc_txn_count,dc_txn_amount,month_avg_balance
0,23791,115269,666373,3191963688.59,716478881.10


In [7]:
%%athena
with recon_base as 
(
select *
    , date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date
from 
    transformations_db.jupiter_reconciliation_transaction_master as t1
where date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) between date('2021-08-01') and date('2022-05-31')
    and
    updated_at = 
        (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
        where t1.reconciliation_id = t2.reconciliation_id
        and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
        and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
        and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
        and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS'))
            )
        )
    and transaction_channel is not null 
)

select * from recon_base limit 3

,reconciliation_id,account_id,bank_cbs_unique_transaction_identifier,payment_unique_identifier,cbs_transaction_datetime,payment_transaction_datetime,transaction_channel,transaction_type,transaction_amount,payments_record_status,reconciliation_status,rrn,jupiter_payment_id,upi_mcc_code,cbs_transaction_fetch_time,created_at,updated_at,customer_id,payments_log_payer_particulars,payments_log_payee_particulars,cbs_log_payer_particulars,cbs_log_payee_particulars,switch_payment_unique_identifier,switch_transaction_datetime,debit_card_txn_type,debit_card_transaction_status,switch_log_payee_particulars,switch_log_payer_particulars,debit_card_msg_type,debit_card_txn_scope,user_saving_account_created_date,user_saving_account_type,full_kyc_completion_date,bank_transfer_type,transactionparticulars2,calling_service_identifier,year,month,day,hour,minute,txn_date
0,8ef2baa0-bce4-4b5b-ab94-a276adfa86f7,77770100349089,1ppfTb8lLZsVwgc+noDiqIh72J/yhq4NC09KJ3o+mW4=,jrn::327aaff0-5135-447b-8634-f9aa30f6dc22:698814702129538,2021-09-28 19:19:28,2021-09-28 19:19:28.993,ADD_MONEY,DEPOSIT,10000.00,SUCCESS,RECONCILIATION_SUCCESSFUL,None,698814702129538,None,2021-09-28 19:19:34.654,2021-09-28 19:18:52.090,2021-09-28 19:19:34.654,327aaff0-5135-447b-8634-f9aa30f6dc22,MR AYURJANAN SENAPATI,AYURJANAN SENAPATI,UPI IN/127111082239/9668200666@jupiteraxis/u/0000,Self,None,None,None,None,None,None,None,None,2021-09-09 12:37:54,jupiter.accounts.full,2021-09-20 17:51:22.145,None,None,None,2021,9,28,2,0,2021-09-28
1,3da56cf0-0535-4ead-89bc-6434a42e211c,77770100329404,MSr8pKsb5Zab7u6odysQ8lWEQrsZDx4UAYd3BjxSAo0=,jrn::d14e86bc-1f94-4e02-baa7-1891d3f09757:369711256120610,2021-09-14 23:21:40,2021-09-14 23:21:40.339,ADD_MONEY,DEPOSIT,400.00,SUCCESS,RECONCILIATION_SUCCESSFUL,None,369711256120610,None,2021-09-14 23:21:42.144,2021-09-14 23:21:31.115,2021-09-14 23:21:42.144,d14e86bc-1f94-4e02-baa7-1891d3f09757,GAUTAM KUMAR ROY,GAUTAM KUMAR RAY,UPI IN/125760906008/8809255485@jupiteraxis/u/0000,Self,None,None,None,None,None,None,None,None,2021-09-07 20:10:21,jupiter.accounts.full,2021-09-10 14:56:57.722,None,None,None,2021,9,14,2,0,2021-09-14
2,99cae658-75d1-442c-96de-7aae6356c963,XXXXXXXXXX15,None,jrn::f985462f-3b04-4f6f-8619-8785f3821fc6:000000410236836,NaT,2021-09-14 06:53:02.319,UPI,WITHDRAWAL,1668.00,SUCCESS,RECONCILIATION_JUPITER_SUCCESS,None,000000410236836,5818,2021-09-14 06:53:02.381,2021-09-14 06:52:18.360,2021-09-14 06:53:02.381,f985462f-3b04-4f6f-8619-8785f3821fc6,SANGALA PRABHAKAR,Dhani,None,None,None,None,None,None,None,None,None,None,2021-09-10 17:17:21,jupiter.accounts.half,NaT,None,None,None,2021,9,14,2,0,2021-09-14


In [13]:
%%athena
drop table if exists dc_txn_dtl

In [14]:
%%athena
create table dc_txn_dtl as 
with recon_base as 
(
select *
    , date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date
from 
    transformations_db.jupiter_reconciliation_transaction_master as t1
where date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) between date('2021-08-01') and date('2022-05-31')
    and
    updated_at = 
        (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
        where t1.reconciliation_id = t2.reconciliation_id
        and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
        and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
        and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
        and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS'))
            )
        )
    and transaction_channel is not null 
)

, base as (
    select customer_id as user_id
    , reconciliation_id as txn_id, txn_date, transaction_channel, transaction_type
    , debit_card_txn_type, debit_card_txn_scope
    , sum(transaction_amount) as txn_amount
    from recon_base
    group by 1,2,3,4,5,6,7
)

select * from base 

In [24]:
%%athena
create table dc_txn_dtl1 as 
with recon_base as 
(
select *
    , date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date
from 
    transformations_db.jupiter_reconciliation_transaction_master as t1
where date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) between date('2021-08-01') and date('2022-05-31')
    and
    updated_at = 
        (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
        where t1.reconciliation_id = t2.reconciliation_id
        and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
        and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
        and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
        and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS'))
            )
        )
    and transaction_channel is not null 
)


, base as (
    select customer_id as user_id
    , reconciliation_id as txn_id, txn_date, transaction_channel, transaction_type
    , (
    case when lower(
      coalesce(
        debit_card_txn_type, cbs_log_payee_particulars
      )
    ) like '%atm%' then 'ATM' else 'Non_ATM' end
  ) as dc_txn_type, 
  (
    case when debit_card_txn_scope is not null then debit_card_txn_scope when transactionparticulars2 like '%/IN' then 'DOMESTIC' when transactionparticulars2 not like '%/IN' then 'INTERNATIONAL' end
  ) as dc_txn_scope 
    , sum(transaction_amount) as txn_amount
    from recon_base
    group by 1,2,3,4,5,6,7
)

select * from base 

In [25]:
%%athena
create table tmp32_1 as 
with gt as (
    select a.*, 'GT100' as flg
    from dc_txn_dtl1 a 
    join user_clv_gt100 b 
    on a.user_id = b.user_id
)

, lt as (
    select a.*, 'LT100' as flg
    from dc_txn_dtl1 a 
    join user_clv_lt100 b 
    on a.user_id = b.user_id
)

select * from(
select * from gt
union
select * from lt
)

In [15]:
%%athena
select * from dc_txn_dtl limit 3

,user_id,txn_id,txn_date,transaction_channel,transaction_type,debit_card_txn_type,debit_card_txn_scope,txn_amount
0,46b43bd8-a717-459f-b57b-5ec4af3f9cc2,fd091400-1e9c-4950-9125-be29c335a5e3,2021-12-05,DEBIT_CARD,WITHDRAWAL,ATM_WITH,DOMESTIC,8500.00
1,7ecb30da-d443-45b7-95e7-eba5fdc942f7,97fa4462-08fc-4965-a15c-d4018dcf843b,2021-12-05,DEBIT_CARD,WITHDRAWAL,POS,DOMESTIC,471.00
2,93e2666d-a417-4a72-af3c-844761b40957,45ba3c72-405a-4221-9546-bc899486d15f,2021-12-05,ADD_MONEY,DEPOSIT,None,None,8000.00


In [17]:
%%athena
create table tmp32 as 
with gt as (
    select a.*, 'GT100' as flg
    from dc_txn_dtl a 
    join user_clv_gt100 b 
    on a.user_id = b.user_id
)

, lt as (
    select a.*, 'LT100' as flg
    from dc_txn_dtl a 
    join user_clv_lt100 b 
    on a.user_id = b.user_id
)

select * from(
select * from gt
union
select * from lt
)

In [18]:
%%athena
select * from tmp32 limit 3

,user_id,txn_id,txn_date,transaction_channel,transaction_type,debit_card_txn_type,debit_card_txn_scope,txn_amount,flg
0,000c7a3a-0092-4185-a420-df559345c6b2,fb52d824-3f81-4177-9f83-a1009503e96f,2021-09-17,DEBIT_CARD,WITHDRAWAL,ATM_WITH,DOMESTIC,1000.00,GT100
1,4107ee33-a04c-4a41-bbac-6c21fa7cc375,cf35c981-1ebf-4169-9434-ebb15e8b2890,2021-12-27,UPI,WITHDRAWAL,None,None,1500.00,GT100
2,2b595a09-f600-4159-9a47-b535411e7c41,c7129639-ada9-47e2-a2ee-cfe9d447fa40,2022-05-26,UPI,WITHDRAWAL,None,None,375.00,GT100


In [23]:
%%athena
with base as (
    select flg
    , date_trunc('month', txn_date) as txn_mo
    , user_id
    , transaction_channel
    , transaction_type
    , debit_card_txn_type
    , debit_card_txn_scope
    , count(distinct txn_id) as txn_count
    , sum(txn_amount) as txn_amount
    from tmp32 
    group by 1,2,3,4,5,6,7
)

select flg
, transaction_type
, count(distinct user_id) as user_count
, sum(txn_mo_count) as txn_mo_count
, sum(txn_count) as txn_count
, sum(txn_amount) as txn_amount
from(
    select flg
    , user_id
    , transaction_type
    , count(distinct txn_mo) as txn_mo_count
    , sum(txn_count) as txn_count
    , sum(txn_amount) as txn_amount
    from base 
    where transaction_channel = 'DEBIT_CARD'
    group by 1,2,3
)
group by 1,2
order by 1,2

,flg,transaction_type,user_count,txn_mo_count,txn_count,txn_amount
0,GT100,DEPOSIT,4313,5036,6368,32757217.92
1,GT100,WITHDRAWAL,23583,85630,665540,3187737895.00
2,LT100,DEPOSIT,25773,29808,42086,36280361.01
3,LT100,WITHDRAWAL,378030,887664,3924023,6176782666.14


In [27]:
%%athena
with base as (
    select flg
    , date_trunc('month', txn_date) as txn_mo
    , user_id
    , transaction_channel
    , transaction_type
    , dc_txn_type
    , dc_txn_scope
    , count(distinct txn_id) as txn_count
    , sum(txn_amount) as txn_amount
    from tmp32_1
    group by 1,2,3,4,5,6,7
)

select flg
, dc_txn_scope
, count(distinct user_id) as user_count
, sum(txn_mo_count) as txn_mo_count
, sum(txn_count) as txn_count
, sum(txn_amount) as txn_amount
from(
    select flg
    , user_id
    , dc_txn_scope
    , count(distinct txn_mo) as txn_mo_count
    , sum(txn_count) as txn_count
    , sum(txn_amount) as txn_amount
    from base 
    where transaction_channel = 'DEBIT_CARD'
    and transaction_type = 'WITHDRAWAL'
    group by 1,2,3
)
group by 1,2
order by 1,2

,flg,dc_txn_scope,user_count,txn_mo_count,txn_count,txn_amount
0,GT100,DOMESTIC,23555,85417,662010,3104065410.84
1,GT100,INTERNATIONAL,733,1096,3530,83672484.16
2,LT100,DOMESTIC,375143,877499,3826727,5852855021.86
3,LT100,INTERNATIONAL,19186,31248,97296,323927644.28


In [28]:
%%athena
select * from user_rwd limit 4

,user_id,txn_date,upi_jewels,dc_jewels,nujo_jewels,activation_jewels,referral_jewels,merchant_jewels,sc_jewels
0,27066860-52d3-4533-adc9-8dad29d6a9b5,2022-03-21,0.00,2.00,0.00,0.00,0.00,0.00,0.00
1,2vP-H8B-GM3O-9EP,2022-05-10,0.00,2.00,0.00,0.00,0.00,0.00,0.00
2,11ccf6fc-41ee-4244-96e7-081938b30049,2022-03-24,0.00,2.00,0.00,0.00,0.00,0.00,0.00
3,e7542693-80d2-4333-a467-6a1f260c9617,2022-05-29,0.00,1.00,0.00,0.00,0.00,0.00,0.00


In [30]:
%%athena
create table user_rwd_mo as 
select user_id, date_trunc('month', txn_date) as txn_mo
, sum(upi_jewels) as upi_jewels
, sum(dc_jewels) as dc_jewels
, sum(nujo_jewels) as nujo_jewels
, sum(activation_jewels) as activation_jewels
, sum(referral_jewels) as referral_jewels
, sum(merchant_jewels) as merchant_jewels
, sum(sc_jewels) as sc_jewels
from user_rwd
group by 1,2

In [32]:
%%athena
create table tmp33 as 
with gt as (
    select a.*, 'GT100' as flg
    from user_rwd_mo a 
    join user_clv_gt100 b 
    on a.user_id = b.user_id
)

, lt as (
    select a.*, 'LT100' as flg
    from user_rwd_mo a 
    join user_clv_lt100 b 
    on a.user_id = b.user_id
)

select * from(
select * from gt
union
select * from lt
)

In [33]:
%%athena
select * from tmp33 limit 3

,user_id,txn_mo,upi_jewels,dc_jewels,nujo_jewels,activation_jewels,referral_jewels,merchant_jewels,sc_jewels,flg
0,fa2113bb-ac0c-4b6c-870a-a67340cb8cd0,2021-11-01,0.00,0.00,0.00,10.00,0.00,0.00,0.00,LT100
1,33c5e044-4d29-4fb1-9ca9-02c1e3455738,2021-09-01,0.00,10.00,0.00,0.00,0.00,0.00,0.00,LT100
2,23e4b785-1288-48fa-986a-3113a713273d,2021-11-01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,LT100


In [34]:
%%athena
select flg
, count(distinct user_id) as user_count
, sum(txn_mo_count) as txn_mo_count
, sum(upi_jewels) as upi_jewels
, sum(dc_jewels) as dc_jewels
, sum(nujo_jewels) as nujo_jewels
, sum(activation_jewels) as activation_jewels
, sum(referral_jewels) as referral_jewels
, sum(merchant_jewels) as merchant_jewels
, sum(sc_jewels) as sc_jewels
from(
    select flg
    , user_id
    , count(distinct txn_mo) as txn_mo_count
    , sum(upi_jewels) as upi_jewels
    , sum(dc_jewels) as dc_jewels
    , sum(nujo_jewels) as nujo_jewels
    , sum(activation_jewels) as activation_jewels
    , sum(referral_jewels) as referral_jewels
    , sum(merchant_jewels) as merchant_jewels
    , sum(sc_jewels) as sc_jewels
    from tmp33
    group by 1,2
)
group by 1
order by 1

,flg,user_count,txn_mo_count,upi_jewels,dc_jewels,nujo_jewels,activation_jewels,referral_jewels,merchant_jewels,sc_jewels
0,GT100,23281,59665,92303.00,996905.00,22859.00,269195.00,25059.00,28836.00,20060.00
1,LT100,1059809,1717145,22284823.17,35912527.00,6514424.00,17890909.00,3134006.00,4273348.00,1018719.00


In [6]:
%%athena
select * from ltv_final1
where media_source in ('None')
limit 5

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,87632bab-fe08-4947-b1fd-1b83a855687a,2022-03-01,2010-01-01,0,None,2022-03-01,2,698.00,158.00,117.50,0.00,0.00,18.00,0.00,0.77,0.00,0.00,2.58,0.00,1.81,18.77,2.58,-16.19
1,10a7c275-0d84-45a8-8c9d-5276d7d8044b,2022-01-01,2010-01-01,0,None,2022-02-01,4,14981.00,0.12,159.44,0.00,0.00,10.00,12.60,1.08,0.00,0.00,55.43,0.00,41.75,23.68,55.43,31.75
2,f3a690d2-308f-4e2b-9882-360f0c93e580,2021-09-01,2010-01-01,0,None,2021-09-01,0,0.00,1.00,1.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00
3,c28fece3-be03-433d-aae8-7673bddc8b9e,2021-11-01,2021-11-01,0,None,2021-12-01,3,828.00,3845.00,4112.80,2.40,2.54,172.00,0.00,0.91,0.00,0.00,3.06,0.00,2.15,172.91,5.60,-167.31
4,a3b36621-50ec-46fd-a3ad-20a37dd416ca,2022-04-01,2010-01-01,0,None,2022-04-01,0,0.00,2563.00,9409.90,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,-10.00


__Credit Score__

In [4]:
%%athena
drop table if exists credit_base

In [5]:
%%athena
create table credit_base as 
select userid ,
max(case when cast(nullif(bureau_score, '') as int) is null or cast(nullif(bureau_score, '') as int)<=0 then 'NTC'
 when cast(nullif(bureau_score, '') as int) between 1 and  6 then 'ETB'
 when cast(nullif(bureau_score, '') as int) between 10 and  600  then '10-600'
 when cast(nullif(bureau_score, '') as int) between 601 and  680  then '601-680'
 when cast(nullif(bureau_score, '') as int) between 681 and  720  then '681-720'
 when cast(nullif(bureau_score, '') as int) > 720 then '720+' end ) as bureau_score_band
 
from (SELECT bureau_score,bureau.userid,
 ROW_NUMBER() OVER (PARTITION BY bureau.userid ORDER BY cast(report_order_date as date)) AS first_row
 
FROM delta_lake_db.credit_bureau_summary  as bureau

join (select prospective_user_id as user_id
      from transformations_db.jupiter_onboarding_master
     where user_saving_account_created_date is not null
     group by 1) as ob
on bureau.userid = ob.user_id  

where product='JUPITER'
  )
where first_row = 1
group by 1

In [6]:
%%athena
select * from credit_base limit 3

,userid,bureau_score_band
0,852bae28-31e0-4476-8f86-455dba7b75b6,NTC
1,70ef0465-254c-4023-916e-3c5a83260b6e,NTC
2,adff694c-1bac-4d33-8eed-9c8d787c325b,681-720


In [8]:
%%athena
select userid, count(*) ac
from credit_base
group by 1 
order by 2 desc 
limit 2

,userid,ac
0,7517e540-a8e7-489d-a983-652d274f5532,1
1,c71b7e7b-f25d-4b44-b942-3382f3451936,1


In [9]:
%%athena
select * from user_clv_gt100 limit 2

,user_id,clv
0,01ffb741-82a4-4ac0-9d27-07b7c822b8f3,267.88
1,a632f534-d6a9-4626-82b0-3802cc42bbb8,104.11


In [12]:
%%athena
with gt100 as (
    select user_id, bureau_score_band
    from user_clv_gt100 a 
    join credit_base b
    on a.user_id = b.userid
    group by 1,2
)

, lt100 as (
    select user_id, bureau_score_band
    from user_clv_lt100 a 
    join credit_base b
    on a.user_id = b.userid
    group by 1,2
)

, base as (
    select 'gt100' as flg, * from gt100
    union
    select 'lt100' as flg, * from lt100    
)

select flg, bureau_score_band, count(distinct user_id) as uc
from base limit 
group by 1,2
order by 1,2

,flg,bureau_score_band,uc
0,gt100,10-600,1756
1,gt100,601-680,1505
2,gt100,681-720,1152
3,gt100,720+,2246
4,gt100,ETB,908
5,gt100,NTC,5329
6,lt100,10-600,60530
7,lt100,601-680,62079
8,lt100,681-720,59744
9,lt100,720+,136897


In [14]:
%%athena
select bureau_score_band, count(distinct userid) as uc
from credit_base
group by 1
order by 1

,bureau_score_band,uc
0,10-600,64476
1,601-680,65930
2,681-720,63426
3,720+,144987
4,ETB,44808
5,NTC,321589


In [16]:
%%athena
select count(distinct user_id)
from user_clv_lt100

,_col0
0,1066647


__PRO split__

In [17]:
%%athena
select * from delta_lake_db.onboarding_public_user_membership_infoset limit 3

,membership_level,aux_created_at,prospective_user_id,aux_updated_at,row_id,aux_raw_user_id
0,l1,2022-06-16 13:28:12.288,3531059,2022-06-16 13:28:12.288,1829619,38b62fda-f1a3-4831-9b42-4d590d6af609
1,l2,2022-06-16 13:36:27.171,3527723,2022-06-16 13:36:27.171,1829675,05ab5b81-b747-4d1f-a750-768e395c57f0
2,l1,2022-06-16 13:44:13.494,3531223,2022-06-16 13:44:13.494,1829717,ce983973-efd8-4267-abe0-b81d07c8cc19


In [19]:
%%athena
create table mem_base as 
select aux_raw_user_id as user_id, membership_level
from delta_lake_db.onboarding_public_user_membership_infoset
where aux_raw_user_id in (select user_id from ltv_final1)
group by 1,2

In [22]:
%%athena
with gt100 as (
    select a.user_id, membership_level
    from user_clv_gt100 a 
    join mem_base b
    on a.user_id = b.user_id
    group by 1,2
)

, lt100 as (
    select a.user_id, membership_level
    from user_clv_lt100 a 
    join mem_base b
    on a.user_id = b.user_id
    group by 1,2
)

, base as (
    select 'gt100' as flg, * from gt100
    union
    select 'lt100' as flg, * from lt100    
)

select flg, membership_level, count(distinct user_id) as uc
from base  
group by 1,2
order by 1,2

,flg,membership_level,uc
0,gt100,l1,8445
1,gt100,l2,14866
2,gt100,l3,480
3,lt100,l1,677218
4,lt100,l2,380640
5,lt100,l3,8789


In [23]:
%%athena
select * from ltv_final1 limit 3

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,8ef2009e-2fa1-478d-ab25-14e320f4b4e1,2021-11-01,2022-05-01,1,Twitter,2022-01-01,1,2130.00,24404.00,16787.87,0.00,0.00,22.00,0.00,2.34,0.00,0.00,7.88,0.00,5.54,24.34,7.88,-16.46
1,ldV-4XX-hz96-Yup,2021-11-01,2021-11-01,0,restricted,2022-01-01,5,1225.00,28.00,50.74,0.02,0.03,10.00,0.00,1.35,0.00,0.00,4.53,0.00,3.19,11.35,4.56,-6.78
2,e13d7dc0-9fca-46c1-973d-70717e33ed03,2022-01-01,2022-01-01,0,onecode_int,2022-03-01,0,0.00,0.00,0.68,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [24]:
%%athena
with l3 as (
    select count(distinct user_id), sum(pf_rev) as pf_rev
    from(
        select a.user_id, sum(monthly_sum_daily_pf_revenue) as pf_rev
        from (select * from 
              mem_base
              where membership_level = 'l3'
             ) a 
        join ltv_final1 b 
        on a.user_id = b.user_id
        group by 1
    )
)

select * from l3

,_col0,pf_rev
0,9269,270955.37


In [26]:
%%athena
with l1_2 as (
    select count(distinct user_id), sum(pf_rev) as pf_rev
    from(
        select a.user_id, sum(monthly_sum_daily_pf_revenue) as pf_rev
        from (select * from 
              mem_base
              where membership_level <> 'l3'
             ) a 
        join ltv_final1 b 
        on a.user_id = b.user_id
        group by 1
    )
)

select * from l1_2

,_col0,pf_rev
0,1081169,1226207.89


__Salary Groups__

In [28]:
%%athena
select * from delta_lake_db.jupiter_user_demographics_data limit 3

,birthyear,communicationaddresscity,communicationaddresspincode,gender,incomeslab,onboardingdate,userid,date,hour
0,1997,KARIMNAGAR,505524,Male,GT1LT3,2022-05-09,c06b0eb6-562f-4478-87d4-07adde1a0ce3,2022-05-09,17
1,2000,Rangareddi,500055,Male,GT0LT1,2021-12-12,46ebd71f-42b7-4c35-b0bc-c3f496e2db8b,2021-12-12,7
2,1999,South 24 Parganas,700039,Male,GT0LT1,2022-03-28,9d8872c7-4839-4bec-8706-383553461f41,2022-03-28,9


In [29]:
%%athena
select userid, count(*)
from delta_lake_db.jupiter_user_demographics_data
group by 1 
order by 2 desc 
limit 2

,userid,_col1
0,86462e69-2026-4953-ac20-9649d50310e1,4
1,11d3458f-59e9-4435-8567-4b3846c11018,4


In [33]:
%%athena
with gt100 as (
    select a.user_id, incomeslab
    from user_clv_gt100 a 
    join (select userid as user_id, max(incomeslab) as incomeslab
          from delta_lake_db.jupiter_user_demographics_data
          group by 1) b
    on a.user_id = b.user_id
    group by 1,2
)

, lt100 as (
    select a.user_id, incomeslab
    from user_clv_lt100 a 
    join (select userid as user_id, max(incomeslab) as incomeslab
          from delta_lake_db.jupiter_user_demographics_data
          group by 1) b
    on a.user_id = b.user_id
    group by 1,2
)

, base as (
    select 'gt100' as flg, * from gt100
    union
    select 'lt100' as flg, * from lt100    
)

select flg, incomeslab, count(distinct user_id) as uc
from base  
group by 1,2
order by 1,2

,flg,incomeslab,uc
0,gt100,GT0LT1,6734
1,gt100,GT100,91
2,gt100,GT10LT15,698
3,gt100,GT15LT25,305
4,gt100,GT1LT3,16
5,gt100,GT1LT5,10216
6,gt100,GT25LT50,230
7,gt100,GT3LT5,22
8,gt100,GT50LT100,84
9,gt100,GT5LT10,2497


__Top Media Source Acquisition__

In [37]:
%%athena
with gt100 as (
    select a.user_id, media_source
    from user_clv_gt100 a 
    join (select user_id, media_source
          from ltv_final1
          group by 1,2) b
    on a.user_id = b.user_id
    group by 1,2
)

, lt100 as (
    select a.user_id, media_source
    from user_clv_lt100 a 
    join (select user_id, media_source
          from ltv_final1
          group by 1,2) b
    on a.user_id = b.user_id
    group by 1,2
)

, base as (
    select 'gt100' as flg, * from gt100
    union
    select 'lt100' as flg, * from lt100    
)

select flg, media_source, count(distinct user_id) as uc
from base 
where flg = 'lt100'
group by 1,2
order by 3 desc 

,flg,media_source,uc
0,lt100,organic,334645
1,lt100,googleadwords_int,138397
2,lt100,restricted,120250
3,lt100,onecode_int,72386
4,lt100,Facebook Ads,71504
5,lt100,None,54698
6,lt100,Partnerships,48581
7,lt100,Virality,36315
8,lt100,phonepe_int,35619
9,lt100,creditkaro_int,29184


__Scenarioe where a user generates CLV >0__

In [38]:
%%athena
select * from ltv_final1 where month_net_clv > 0 
limit 5

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,7541a842-d2ad-4ba1-9b29-7a9e1069b85c,2022-04-01,2022-05-01,1,organic,2022-04-01,7,25500.00,92.00,567.62,0.00,0.00,10.00,22.95,0.00,0.00,0.00,94.35,0.00,71.40,32.95,94.35,61.40
1,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2022-02-01,0,0.00,500.60,222.56,0.31,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.13,0.13
2,df6ade23-1af2-43bb-b764-ad74193ad8f3,2022-02-01,2022-02-01,0,organic,2022-03-01,0,0.00,0.26,227.77,0.00,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.15,0.15
3,uO2-LFK-kure-w5u,2021-07-01,2021-08-01,0,None,2022-01-01,1,150.00,4505.83,5600.99,2.82,3.57,2.00,0.00,0.17,0.00,0.00,0.56,0.00,0.39,2.17,4.12,1.96
4,02a56aca-a4ee-408f-9e18-983032654039,2021-11-01,2022-02-01,0,restricted,2022-03-01,6,10310.00,641.21,270.60,0.40,0.17,9.00,8.55,0.89,0.00,0.00,38.15,0.00,28.71,18.44,38.32,19.88


In [39]:
%%athena
select * from ltv_final1
where user_id = '7541a842-d2ad-4ba1-9b29-7a9e1069b85c'
order by ltv_month

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,7541a842-d2ad-4ba1-9b29-7a9e1069b85c,2022-04-01,2022-05-01,1,organic,2022-04-01,7,25500.00,92.00,567.62,0.00,0.00,10.00,22.95,0.00,0.00,0.00,94.35,0.00,71.40,32.95,94.35,61.40
1,7541a842-d2ad-4ba1-9b29-7a9e1069b85c,2022-04-01,2022-05-01,1,organic,2022-05-01,7,22905.00,7.00,117.00,0.00,0.01,5.00,20.25,0.45,0.00,0.00,84.75,0.00,64.05,25.70,84.76,59.06


In [40]:
%%athena
select * from ltv_final1
where user_id = '2d94f794-541f-4b0b-b034-6f667d86c85a'
order by ltv_month

,user_id,ob_month,fkyc_month,fkyc_flag,media_source,ltv_month,dc_txn_count,dc_txn_amount,eom_balance,month_avg_balance,eom_balance_pf_revenue,monthly_sum_daily_pf_revenue,total_rewards_cost,domestic_atm_cost,domestic_non_atm_cost,intl_atm_cost,intl_non_atm_cost,domestic_interchange_revenue,intl_interchange_revenue,dc_clv,month_net_expense,month_net_revenue,month_net_clv
0,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,1,organic,2021-11-01,0,0.00,51.00,158.83,0.03,0.03,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,0.03,-19.97
1,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2021-12-01,1,100.40,1351.60,734.39,0.84,0.45,1.00,0.00,0.11,0.00,0.00,0.37,0.00,0.26,1.11,0.82,-0.29
2,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2022-01-01,0,0.00,51.60,667.73,0.03,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.43,0.43
3,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2022-02-01,0,0.00,500.60,222.56,0.31,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.13,0.13
4,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2022-03-01,0,0.00,3273.60,789.89,2.05,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.50
5,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2022-04-01,0,0.00,870.60,842.03,0.54,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.52,0.52
6,2d94f794-541f-4b0b-b034-6f667d86c85a,2021-11-01,2021-11-01,0,organic,2022-05-01,1,149.00,61.60,89.02,0.04,0.06,2.00,0.00,0.16,0.00,0.00,0.55,0.00,0.39,2.16,0.61,-1.56
